In [1]:
import re    #正则表达式，进行文字匹配
import urllib   #获取网页数据
import xlwt  #进行excel操作
from bs4 import BeautifulSoup   #网页解析，获取数据

# 获取数据

In [2]:
baseurl = "https://movie.douban.com/top250?start="

head = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36"
}#模拟浏览器头部信息

html_list = []

for i in range(10):                                                   #获取十次网页内容
    url = baseurl+str(i*25)
    request = urllib.request.Request(url,headers=head)                #发送请求
    try:
        response = urllib.request.urlopen(request,timeout=5)          #获取反应
        html=response.read().decode("utf-8")                          #获取网页内容
        html_list.append(html)
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
              print(e.code)
        if hasattr(e,"reason"):
              print(e.reason)

# 数据解析

In [3]:
data_list = []
for i in range(10):
    soup = BeautifulSoup(html_list[i],"html.parser")
    for item in soup.find_all("div",class_="item"):
        data=[]
        item=str(item)
        title=re.findall(re.compile(r'<span class="title">(.*)</span>'),item)[0]#添加电影标题
        data.append(title)

        inf=re.findall(re.compile(r'<p class="">(.*?)</p>',re.S),item)[0]#获取导演与年份的相关信息
        
        director=re.findall(re.compile(r'导演: ([\u4e00-\u9fa5]{0,}.[\u4e00-\u9fa5]{0,})'),inf)[0]#添加电影导演
        data.append(director)
        
        year=re.findall(re.compile(r'(\d{4})'),inf)[0]#添加电影年份
        data.append(year)
        
        rating=re.findall(re.compile(r'<span class="rating_num" property="v:average">(.*)</span>'),item)[0]#添加电影评分
        data.append(rating)
        
        people=re.findall(re.compile(r'<span>(\d*)人评价</span>'),item)[0]#添加评价人数
        data.append(people)
        
        inq=re.findall(re.compile(r'<span class="inq">(.*)</span>'),item)#添加剧情介绍
        if len(inq)==0 :
            data.append(" ")
        else:
            data.append(inq[0])
        data_list.append(data)

In [4]:
data_list[0]

['肖申克的救赎', '弗兰克·德拉邦特', '1994', '9.7', '2334782', '希望让人自由。']

# 保存数据

In [5]:
#将数据保存至excel，再从navicat导入excel文件存入数据库

book = xlwt.Workbook(encoding="utf-8",style_compression=0)
sheet=book.add_sheet('豆瓣电影TOP250',cell_overwrite_ok=True)
col=('电影标题','导演','上映年份','评分','评价人数','剧情介绍')
for j in range(6):
    sheet.write(0,j,col[j])
for i in range(250):
    for j in range(6):
        sheet.write(i+1,j,data_list[i][j])
book.save('douban.xls')